# Sketch Classification

In [ ]:
This notebook is inspired by Google's Quick Draw Project: 
- https://quickdraw.withgoogle.com/
Data source: 
- https://github.com/googlecreativelab/quickdraw-dataset
The model is deployed at:
- http://35.237.138.188

## Shuffle Data

In [ ]:
import os
import re
import pickle
import random
from memory_map import MemoryMap

In [ ]:
INPUT_PATH = './train_simplified'
N_LABEL_SAMPLE = 50000
VALIDATION_SIZE = 20000

In [ ]:
def read_offsets():
    files = os.listdir(INPUT_PATH)
    return [f.split('.')[0] for f in files if re.search('\.offsets$', f)]

In [ ]:
filenames = read_offsets()
print(len(filenames))

In [ ]:
memmaps = []
for index, filename in enumerate(filenames):
    memmaps.append(MemoryMap(INPUT_PATH, filename))

In [ ]:
metadata = []
for index, filename in enumerate(filenames):
    file_metadata_path = os.path.join(INPUT_PATH, filename + ".offsets")
    with open (file_metadata_path, 'rb') as fp:
        offsets = pickle.load(fp)
        metadata.extend([(index,) + offset for offset in offsets[:N_LABEL_SAMPLE]])

In [ ]:
def read_line(line_pointer):
    (file_index, start, end) = line_pointer
    return memmaps[file_index].memmap[start:end-1]

In [ ]:
shuffled = metadata[:]
random.shuffle(shuffled)
read_line(shuffled[0])

In [ ]:
len(shuffled)

In [ ]:
train_offsets = shuffled[:-VALIDATION_SIZE]
val_offsets = shuffled[-VALIDATION_SIZE:]

In [ ]:
len(train_offsets)

In [ ]:
len(val_offsets)

## Prepare Data

In [ ]:
import cv2
import json
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.mobilenet import preprocess_input


In [ ]:
np.random.seed(1988)
tf.set_random_seed(1988)

In [ ]:
N_LABELS = 340
STEPS = 20
SIZE = 64
BASE_SIZE = 256
BATCH_SIZE = 1000
EPOCHS = 90

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(filenames)

I tried different drawing techniques such as creating rgb drawing where each stroke has a unique color and creating a grayscale drawing where the first stroke has black and it gets lighter as you go through more strokes. The RGB drawings took much longer to train and didn't have much affect on the accuracy, so I decided to work with grayscale drawings. The below function does something similar to the stroke by stroke grayscale drawing, but the color gets lighter as you go through pixels. This method gave me the best results.

I am using the simplified version of the training data where the time information is stripped out and the strokes are simplified. Here is the process of simplification: 

1- Strip time information
2- Align the drawing to the top-left corner, to have minimum values of 0
3- Uniformly scale the drawing, to have a maximum value of 255
4- Resample all strokes with a 1 pixel spacing
5- Simplify all strokes using the Ramer–Douglas–Peucker algorithm with an epsilon value of 2.0

The accuracy of the model could be improved slightly if I used the time information and draw the images using the time information. However, the strokes are ordered in the simplified version as well. Using the pixel ordering as "time" information gave pretty good results. You can manually test the results on the website I deployed for sketch classification. http://35.237.138.188 The model can predict the sketches pretty early on.

In [ ]:
#https://gist.github.com/hasibzunair/5ea9009a28e4c5a8e6e44bafe6ba4104
def draw(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

In [ ]:
def image_generator(size, batchsize, lw=6, time_color=True):
    while True:
        for i in range(len(train_offsets) // batchsize + (len(train_offsets) % batchsize > 0)):
            batch = train_offsets[i*batchsize:min((i+1)*batchsize,len(train_offsets))]
            x = np.zeros((batchsize, size, size, 1))
            y = []
            for i in range(batchsize):
                line = read_line(batch[i]).decode(encoding="utf-8")
                if line != '':
                    raw_strokes = ast.literal_eval(re.findall(r'"(.*?)"', line)[0])
                    x[i, :, :, 0] = draw(raw_strokes, size=size, lw=6, time_color=True)
                    y.append(line.rsplit(',', 1)[1])
            x = preprocess_input(x).astype(np.float32)
            y = to_categorical(label_encoder.transform(y), num_classes=N_LABELS)
            yield x,y

In [ ]:
x_valid = np.zeros((VALIDATION_SIZE, SIZE, SIZE, 1))
y = []
for i in range(VALIDATION_SIZE):
    line = read_line(val_offsets[i]).decode(encoding="utf-8")
    if line != '':
        raw_strokes = ast.literal_eval(re.findall(r'"(.*?)"', line)[0])
        x_valid[i, :, :, 0] = draw(raw_strokes, size=SIZE, lw=6, time_color=True)
        y.append(line.rsplit(',', 1)[1])  
x_valid = preprocess_input(x_valid).astype(np.float32)
y_valid = to_categorical(label_encoder.transform(y), num_classes=N_LABELS)
print(x_valid.shape, y_valid.shape)

In [ ]:
train_datagen = image_generator(size=SIZE, batchsize=BATCH_SIZE)

In [ ]:
%%timeit
x, y = next(train_datagen)

In [ ]:
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

## Model

In [ ]:
model = MobileNet(input_shape=(SIZE, SIZE, 1), alpha=1., weights=None, classes=N_LABELS)
model.compile(optimizer=Adam(lr=0.0015), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

In [ ]:
callbacks = [
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.75, patience=5, min_delta=0.001, mode='max', min_lr=1e-5, verbose=1),
    ModelCheckpoint('model.h5', monitor='val_top_3_accuracy', mode='max', save_best_only=True, save_weights_only=True),
]

In [ ]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)

In [ ]:
hist_df = pd.concat([pd.DataFrame(hist.history) for hist in hists], sort=True)
hist_df.index = np.arange(1, len(hist_df)+1)
fig, axs = plt.subplots(nrows=1, sharex=True, figsize=(16, 10))
axs.plot(hist_df.val_categorical_accuracy, lw=5, label='Validation Accuracy')
axs.plot(hist_df.categorical_accuracy, lw=5, label='Training Accuracy')
axs.set_ylabel('Accuracy')
axs.set_xlabel('Epoch')
axs.grid()
axs.legend(loc=0)
fig.savefig('Accuracy.png', dpi=500)
plt.show();

In [ ]:
hist_df = pd.concat([pd.DataFrame(hist.history) for hist in hists], sort=True)
hist_df.index = np.arange(1, len(hist_df)+1)
fig, axs = plt.subplots(nrows=1, sharex=True, figsize=(16, 10))
axs.plot(hist_df.val_categorical_crossentropy, lw=4, label='Validation MLogLoss')
axs.plot(hist_df.categorical_crossentropy, lw=4, label='Training MLogLoss')
axs.set_ylabel('MLogLoss')
axs.set_xlabel('Epoch')
axs.grid()
axs.legend(loc=0)
fig.savefig('MLogLoss.png', dpi=500)
plt.show();